In [58]:
# 1 check functionality of environment
print("Hola mundo")
print("Entorno reactivado correctamente")


Hola mundo
Entorno reactivado correctamente


In [59]:
# 2 cargar librerias
!pip install pyxlsb

In [60]:
# 3 cargar librerias
!pip install unidecode


In [61]:
# 4 cargar librerias

import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import unicodedata


In [62]:
# 5 download data 

# Leer todas las hojas del archivo de una vez
print("Cargando archivo... esto puede tomar unos minutos")
excel_file = pd.ExcelFile('../tlm_conagua_2012_2024.xlsb', engine='pyxlsb')

# Cargar las hojas específicas
df_sitios = excel_file.parse('Sitios')
df_resultados = excel_file.parse('Resultados')

print("¡Archivo cargado exitosamente!")

Cargando archivo... esto puede tomar unos minutos
¡Archivo cargado exitosamente!


In [63]:
# 6 explorar los datos

# Verificar las dimensiones
print("Sitios:", df_sitios.shape)
print("Resultados:", df_resultados.shape)

# Ver las primeras filas y columnas
print("\nColumnas en Sitios:")
print(df_sitios.columns.tolist())

print("\nColumnas en Resultados:")
print(df_resultados.columns.tolist())

# Verificar los nombres exactos de las columnas clave
columnas_comunes = ['CLAVE SITIO', 'NOMBRE DEL SITIO', 'TIPO DE CUERPO DE AGUA']
print()

# Verificar si existen en ambos DataFrames
for col in columnas_comunes:
    print(f"'{col}' en Sitios: {col in df_sitios.columns}")
    print(f"'{col}' en Resultados: {col in df_resultados.columns}")

Sitios: (7771, 14)
Resultados: (129235, 469)

Columnas en Sitios:
['CLAVE SITIO', 'NOMBRE DEL SITIO', 'CUENCA', 'CLAVE ACUÍFERO', 'ACUÍFERO', 'ORGANISMO CUENCA', 'DIRECCIÓN LOCAL', 'ESTADO', 'MUNICIPIO', 'CUERPO DE AGUA', 'TIPO DE CUERPO DE AGUA', 'SUBTIPO CUERPO AGUA', 'LATITUD', 'LONGITUD']

Columnas en Resultados:
['CLAVE SITIO', 'CLAVE DE MONITOREO', 'NOMBRE DEL SITIO', 'TIPO CUERPO DE AGUA', 'FECHA REALIZACIÓN', 'Año', 'ALC_FEN', 'ALC_TOT', 'CO3', 'HCO3', 'OH', 'CLOROF_A', 'COLI_FEC', 'COLI_TOT', 'E_COLI', 'COT', 'COT_SOL', 'DBO_SOL', 'DBO_TOT', 'DQO_SOL', 'DQO_TOT', 'N_NH3', 'N_NO2', 'N_NO3', 'N_ORG', 'N_TOT', 'N_TOTK', 'TOX_D_48_UT', 'TOX_D_48_SUP_UT', 'TOX_D_48_FON_UT', 'TOX_FIS_SUP_15_UT', 'TOX_FIS_SUP_30_UT', 'TOX_FIS_SUP_5_UT', 'TOX_FIS_FON_15_UT', 'TOX_FIS_FON_30_UT', 'TOX_FIS_FON_5_UT', 'TOX_V_15_UT', 'TOX_V_30_UT', 'TOX_V_5_UT', 'AOXF', 'AOXP', 'AOXT', 'P_TOT', 'PO4_TOT', 'ORTO_PO4', 'COLOR_VER', 'TRANSPARENCIA', 'ABS_UV', 'SDT', 'CONDUC_CAMPO', 'CONDUC_CAMPO_SUP', 'CONDU

In [64]:
# 7 merge de las 2 hojas de conagua

# Primero, identifica qué columnas quieres agregar
# tipo de cuerpo de agua en Sitios 'TIPO DE CUERPO DE AGUA'
# tipo cuerpo de agua en Resultados 'TIPO CUERPO DE AGUA'

# no quiero importar 'NOMBRE DEL SITIO', 'ORGANISMO CUENCA', 'DIRECCIÓN LOCAL' de sitios a resultados
columnas_a_agregar = [col for col in df_sitios.columns 
                     if col not in ['NOMBRE DEL SITIO', 'ORGANISMO CUENCA', 'DIRECCIÓN LOCAL']]

# merge el resto de las columnas de sitios a resultados usando 'CLAVE SITIO' como clave
df_final = df_resultados.merge(df_sitios[columnas_a_agregar], 
                              on='CLAVE SITIO', 
                              how='left')

# explorar el resultado del merge
print(df_final.head())
print(df_final.shape)
list(df_final.columns)
df_final.info()

       CLAVE SITIO      CLAVE DE MONITOREO     NOMBRE DEL SITIO  \
0  BROTE CARMINA 3  BROTE CARMINA 3-131122      CARMINA 3 BROTE   
1        CARMINA 2        CARMINA 2-131122            CARMINA 2   
2         CAZEPA-1         CAZEPA-1-260921  POZO SAN FERNANDO 1   
3         CAZEPA-1         CAZEPA-1-310722  POZO SAN FERNANDO 1   
4         CAZEPA-1         CAZEPA-1-181023  POZO SAN FERNANDO 1   

  TIPO CUERPO DE AGUA FECHA REALIZACIÓN   Año ALC_FEN      ALC_TOT  CO3  \
0         SUBTERRÁNEO             44873  2022     NaN 139.70000000  NaN   
1              LÓTICO             44873  2022     NaN          NaN  NaN   
2         SUBTERRÁNEO             44462  2021     NaN 148.64500000    0   
3         SUBTERRÁNEO             44769  2022     NaN 141.30000000  NaN   
4         SUBTERRÁNEO             45217  2023       0 210.24000000    0   

          HCO3  ...       CUENCA CLAVE ACUÍFERO          ACUÍFERO  \
0          NaN  ...          NaN   522.00000000  PRESA LA AMISTAD   
1       

In [65]:
# 8 limpiar 'tipo cuerpo de agua' en Resultados y 'tipo de cuerpo de agua' en Sitios 

# Ver valores únicos de cada variable
print("Valores únicos en 'TIPO CUERPO DE AGUA':")
print(df_final['TIPO CUERPO DE AGUA'].unique())
print('tipo cuerpo de agua resultados valores', df_final['TIPO CUERPO DE AGUA'].nunique())

print("\n" + "="*50 + "\n")

print("Valores únicos en 'TIPO DE CUERPO DE AGUA':")
print(df_final['TIPO DE CUERPO DE AGUA'].unique())
print('tipo de cuerpo de agua sitios valores', df_final['TIPO DE CUERPO DE AGUA'].nunique())

print("\n" + "="*50 + "\n")




Valores únicos en 'TIPO CUERPO DE AGUA':
['SUBTERRÁNEO' 'LÓTICO' 'LÉNTICO' 'LÉNTICO (HUMEDAL)' 'LÓTICO (HUMEDAL)'
 'COSTERO' 'COSTERO (HUMEDAL)' 'LÓTICO - COSTERO'
 'LÓTICO - COSTERO (HUMEDAL)' 'LÉNTICO - COSTERO'
 'LÉNTICO - COSTERO (HUMEDAL)' 'LÓTICO (ESTUDIO ESPECIAL)'
 'SUBTERRÁNEO (ESTUDIO ESPECIAL)' 'LÉNTICO (ESTUDIO ESPECIAL)'
 'SUBTERRÁNEO (HUMEDAL)' 'LÓTICO Y SEDIMENTOS' 'COSTERO Y SEDIMENTOS'
 'LOTICO ' 'LÉNTICO ' 'LOTICO' 'LÓTICO ']
tipo cuerpo de agua resultados valores 21


Valores únicos en 'TIPO DE CUERPO DE AGUA':
['SUBTERRÁNEO' 'LÓTICO' 'LÉNTICO' 'LÉNTICO (HUMEDAL)' 'LÓTICO (HUMEDAL)'
 'COSTERO' 'COSTERO (HUMEDAL)' 'LÓTICO - COSTERO'
 'LÓTICO - COSTERO (HUMEDAL)' 'LÉNTICO - COSTERO'
 'LÉNTICO - COSTERO (HUMEDAL)' 'LÓTICO (ESTUDIO ESPECIAL)'
 'SUBTERRÁNEO (ESTUDIO ESPECIAL)' 'LÉNTICO (ESTUDIO ESPECIAL)'
 'SUBTERRÁNEO (HUMEDAL)' 'LÓTICO Y SEDIMENTOS' 'COSTERO Y SEDIMENTOS'
 'LÓTICO ' 'LÉNTICO ' 'SUBTERRANEO']
tipo de cuerpo de agua sitios valores 20




In [66]:
# 9 Ver conteos para cada variable tipo cuerpo de agua de resultados
print("Conteos en 'TIPO CUERPO DE AGUA':")
print(df_final['TIPO CUERPO DE AGUA'].value_counts())

print("\n" + "="*50 + "\n")



Conteos en 'TIPO CUERPO DE AGUA':
TIPO CUERPO DE AGUA
LÓTICO                            55807
LÓTICO (HUMEDAL)                  14345
SUBTERRÁNEO                       13868
COSTERO (HUMEDAL)                 13505
COSTERO                            9976
LÉNTICO (HUMEDAL)                  9373
LÉNTICO                            4872
LÓTICO - COSTERO (HUMEDAL)         2726
LÓTICO - COSTERO                   2251
LÉNTICO - COSTERO (HUMEDAL)        1388
LÉNTICO (ESTUDIO ESPECIAL)          296
LÉNTICO - COSTERO                   267
LÓTICO (ESTUDIO ESPECIAL)           214
SUBTERRÁNEO (ESTUDIO ESPECIAL)      196
LOTICO                               71
COSTERO Y SEDIMENTOS                 36
SUBTERRÁNEO (HUMEDAL)                12
LÓTICO                               12
LÉNTICO                              11
LÓTICO Y SEDIMENTOS                   8
LOTICO                                1
Name: count, dtype: int64




In [67]:
# 10 examinar tipo de cuerpo de agua de la hoja de sitios

print("Conteos en 'TIPO DE CUERPO DE AGUA':")
print(df_final['TIPO DE CUERPO DE AGUA'].value_counts())
print("\n" + "="*50 + "\n")

Conteos en 'TIPO DE CUERPO DE AGUA':
TIPO DE CUERPO DE AGUA
LÓTICO                            55808
LÓTICO (HUMEDAL)                  14345
SUBTERRÁNEO                       13866
COSTERO (HUMEDAL)                 13505
COSTERO                            9976
LÉNTICO (HUMEDAL)                  9383
LÉNTICO                            4869
LÓTICO - COSTERO (HUMEDAL)         2726
LÓTICO - COSTERO                   2251
LÉNTICO - COSTERO (HUMEDAL)        1388
LÉNTICO (ESTUDIO ESPECIAL)          296
LÉNTICO - COSTERO                   267
LÓTICO (ESTUDIO ESPECIAL)           214
SUBTERRÁNEO (ESTUDIO ESPECIAL)      196
LÓTICO                               83
COSTERO Y SEDIMENTOS                 36
SUBTERRÁNEO (HUMEDAL)                12
LÓTICO Y SEDIMENTOS                   8
LÉNTICO                               4
SUBTERRANEO                           2
Name: count, dtype: int64




In [68]:
# 11 limpiar 'tipo cuerpo de agua' en Resultados  

# Crear una copia de la variable original
# df_final['tipo_cuerpo_agua_limpio'] = df_final['TIPO CUERPO DE AGUA'].copy()

# Convertir todo a minúsculas y eliminar acentos
df_final['tipo_cuerpo_agua_limpio'] = df_final['TIPO CUERPO DE AGUA'].str.lower().apply(unidecode)

# eliminar acentos
#df_final['tipo_cuerpo_agua_limpio'] = df_final['tipo_cuerpo_agua_limpio'].apply(unidecode)

# Quitar espacios entre palabras 
df_final['tipo_cuerpo_agua_limpio'] = df_final['tipo_cuerpo_agua_limpio'].str.replace(' ', '', regex=False)

# averiguar los resultados  y Comparar original vs limpio [:10]
print("Valores únicos originales:")
print(df_final['TIPO DE CUERPO DE AGUA'].value_counts())
print("\n" + "="*50 + "\n")
print("\nValores únicos limpios:")
print(df_final['tipo_cuerpo_agua_limpio'].value_counts())

Valores únicos originales:
TIPO DE CUERPO DE AGUA
LÓTICO                            55808
LÓTICO (HUMEDAL)                  14345
SUBTERRÁNEO                       13866
COSTERO (HUMEDAL)                 13505
COSTERO                            9976
LÉNTICO (HUMEDAL)                  9383
LÉNTICO                            4869
LÓTICO - COSTERO (HUMEDAL)         2726
LÓTICO - COSTERO                   2251
LÉNTICO - COSTERO (HUMEDAL)        1388
LÉNTICO (ESTUDIO ESPECIAL)          296
LÉNTICO - COSTERO                   267
LÓTICO (ESTUDIO ESPECIAL)           214
SUBTERRÁNEO (ESTUDIO ESPECIAL)      196
LÓTICO                               83
COSTERO Y SEDIMENTOS                 36
SUBTERRÁNEO (HUMEDAL)                12
LÓTICO Y SEDIMENTOS                   8
LÉNTICO                               4
SUBTERRANEO                           2
Name: count, dtype: int64



Valores únicos limpios:
tipo_cuerpo_agua_limpio
lotico                          55891
lotico(humedal)                 143

In [69]:
# 12 crear funcion para simplificar tipos cuerpo de agua
def simplificar_tipo_cuerpo(valor):
    """
    Simplifica las categorías de tipo de cuerpo de agua
    """
    if pd.isna(valor):  # Manejar valores nulos
        return valor
    
    # Convertir a string por si acaso
    valor = str(valor).lower()
    
    # Eliminar especificaciones entre paréntesis
    # .split('(') - Divide el texto usando el paréntesis '(' como separador
    # [0] - Toma solo la primera parte (antes del paréntesis)
    valor = valor.split('(')[0]
    
    # Eliminar "ysedimentos"
    valor = valor.replace('ysedimentos', '')
    
    # Limpiar espacios extra
    valor = valor.strip()
    
    return valor

# Crear la nueva variable simplificada
df_final['tipo_cuerpo_simplificado'] = df_final['tipo_cuerpo_agua_limpio'].apply(simplificar_tipo_cuerpo)

# Ver los nuevos valores únicos y sus conteos
print("hay 6 valores únicos después de simplificar:")
print(df_final['tipo_cuerpo_simplificado'].value_counts())

hay 6 valores únicos después de simplificar:
tipo_cuerpo_simplificado
lotico             70458
costero            23517
lentico            14552
subterraneo        14076
lotico-costero      4977
lentico-costero     1655
Name: count, dtype: int64


In [70]:
# 13 limpiar 'tipo de cuerpo de agua' en Sitios 

def simplificar_tipo_cuerpo(valor):
    """
    Limpia y simplifica las categorías de tipo de cuerpo de agua
    Combina todos los pasos: minúsculas, eliminar acentos, espacios y paréntesis
    """
    if pd.isna(valor):  # Manejar valores nulos
        return valor
    
    # Convertir a string y minúsculas
    valor = str(valor).lower()
    
    # Eliminar acentos
    valor = unidecode(valor)
    
    # Eliminar espacios entre palabras
    valor = valor.replace(' ', '')
    
    # Eliminar especificaciones entre paréntesis
    valor = valor.split('(')[0]
    
    # Eliminar "ysedimentos"
    valor = valor.replace('ysedimentos', '')
    
    # Limpiar espacios extra (por si quedaron)
    valor = valor.strip()
    
    return valor

# Aplicar toda la limpieza en una sola línea
df_final['tipo_de_cuerpo_simplificado'] = df_final['TIPO DE CUERPO DE AGUA'].apply(simplificar_tipo_cuerpo)

# averiguar los resultados  y Comparar original vs limpio [:10]
print("Valores únicos originales tipo de cuerpo de Agua de Sitios:")
print(df_final['TIPO DE CUERPO DE AGUA'].value_counts())

print("\n" + "="*50 + "\n")
# Ver los nuevos valores únicos y sus conteos
print("\nHay 6 valores de tipo de cuerpo de agua de SITIOS despues de simplificar:")
print(df_final['tipo_de_cuerpo_simplificado'].value_counts())

print("\n" + "="*50 + "\n")

# compararar con tipo cuerpo de agua de resultados
print("hay 6 valores de tipo cuerpo de agua de Resultados después de simplificar:")
print(df_final['tipo_de_cuerpo_simplificado'].value_counts())

Valores únicos originales tipo de cuerpo de Agua de Sitios:
TIPO DE CUERPO DE AGUA
LÓTICO                            55808
LÓTICO (HUMEDAL)                  14345
SUBTERRÁNEO                       13866
COSTERO (HUMEDAL)                 13505
COSTERO                            9976
LÉNTICO (HUMEDAL)                  9383
LÉNTICO                            4869
LÓTICO - COSTERO (HUMEDAL)         2726
LÓTICO - COSTERO                   2251
LÉNTICO - COSTERO (HUMEDAL)        1388
LÉNTICO (ESTUDIO ESPECIAL)          296
LÉNTICO - COSTERO                   267
LÓTICO (ESTUDIO ESPECIAL)           214
SUBTERRÁNEO (ESTUDIO ESPECIAL)      196
LÓTICO                               83
COSTERO Y SEDIMENTOS                 36
SUBTERRÁNEO (HUMEDAL)                12
LÓTICO Y SEDIMENTOS                   8
LÉNTICO                               4
SUBTERRANEO                           2
Name: count, dtype: int64



Hay 6 valores de tipo de cuerpo de agua de SITIOS despues de simplificar:
tipo_de_cuerpo

In [71]:
# 14 
df_final.columns.tolist()
# 3 news vars  'tipo_cuerpo_agua_limpio',  'tipo_cuerpo_simplificado',  'tipo_de_cuerpo_simplificado'
# need to del 'tipo_cuerpo_agua_limpio'
df_final.drop(columns=['tipo_cuerpo_agua_limpio'], inplace=True) 
df_final.columns.tolist()   

['CLAVE SITIO',
 'CLAVE DE MONITOREO',
 'NOMBRE DEL SITIO',
 'TIPO CUERPO DE AGUA',
 'FECHA REALIZACIÓN',
 'Año',
 'ALC_FEN',
 'ALC_TOT',
 'CO3',
 'HCO3',
 'OH',
 'CLOROF_A',
 'COLI_FEC',
 'COLI_TOT',
 'E_COLI',
 'COT',
 'COT_SOL',
 'DBO_SOL',
 'DBO_TOT',
 'DQO_SOL',
 'DQO_TOT',
 'N_NH3',
 'N_NO2',
 'N_NO3',
 'N_ORG',
 'N_TOT',
 'N_TOTK',
 'TOX_D_48_UT',
 'TOX_D_48_SUP_UT',
 'TOX_D_48_FON_UT',
 'TOX_FIS_SUP_15_UT',
 'TOX_FIS_SUP_30_UT',
 'TOX_FIS_SUP_5_UT',
 'TOX_FIS_FON_15_UT',
 'TOX_FIS_FON_30_UT',
 'TOX_FIS_FON_5_UT',
 'TOX_V_15_UT',
 'TOX_V_30_UT',
 'TOX_V_5_UT',
 'AOXF',
 'AOXP',
 'AOXT',
 'P_TOT',
 'PO4_TOT',
 'ORTO_PO4',
 'COLOR_VER',
 'TRANSPARENCIA',
 'ABS_UV',
 'SDT',
 'CONDUC_CAMPO',
 'CONDUC_CAMPO_SUP',
 'CONDUC_CAMPO_MED',
 'CONDUC_CAMPO_FON',
 'pH_CAMPO',
 'pH_CAMPO_SUP',
 'pH_CAMPO_MED',
 'pH_CAMPO_FON',
 'SALINIDAD_SUP',
 'SALINIDAD_MED',
 'SALINIDAD_FON',
 'OD_%',
 'OD_mg/L',
 'OD_%_SUP',
 'OD_mg/L_SUP',
 'OD_%_MED',
 'OD_mg/L_MED',
 'OD_%_FON',
 'OD_mg/L_FON',
 '1,1,1

In [72]:
# 15 eliminar vars

# Definir las columnas que quieres mantener
columnas_deseadas_s = ['CLAVE SITIO',
                    'CLAVE DE MONITOREO',
                    'NOMBRE DEL SITIO',
                    'TIPO CUERPO DE AGUA',
                    'tipo_cuerpo_simplificado',
                    'FECHA REALIZACIÓN',
                    'Año',
                    'CUENCA',
                    'CLAVE ACUÍFERO',
                    'ACUÍFERO',
                    'ESTADO',
                    'MUNICIPIO',
                    'CUERPO DE AGUA',
                    'TIPO DE CUERPO DE AGUA',
                    'tipo_de_cuerpo_simplificado',
                    'SUBTIPO CUERPO AGUA',
                    'LATITUD',
                    'LONGITUD',
                    'PROFUNDIDAD',
                    'CAUDAL',
                    'FLUORUROS_SOL',
                    'FLUORUROS_TOT',
                    'AS_TOT',
                    'E_COLI_UFC',
                    'COLI_FEC_UFC',
                    'COLI_TOT_UFC',
                    'COLI_FEC',
                    'COLI_TOT',
                    'E_COLI']

# Crear el DataFrame reducido
df_s= df_final[columnas_deseadas_s].copy()
print(df_s.shape)
print()
# def las columnas deseadas para el df_m
columnas_deseadas_m = ['CLAVE SITIO',
                      'CLAVE DE MONITOREO',
                      'NOMBRE DEL SITIO',
                      'TIPO CUERPO DE AGUA',
                      'tipo_cuerpo_simplificado',
                      'FECHA REALIZACIÓN',
                      'Año',
                      'CUENCA',
                      'CLAVE ACUÍFERO',
                      'ACUÍFERO',
                      'ESTADO',
                      'MUNICIPIO',
                      'CUERPO DE AGUA',
                      'TIPO DE CUERPO DE AGUA',
                      'tipo_de_cuerpo_simplificado',
                      'SUBTIPO CUERPO AGUA',
                      'LATITUD',
                      'LONGITUD',
                      'ACID_TOT',
                      'AS_SOL',
                      'AS_TOT',
                      'CAUDAL',
                      'CD_SOL',
                      'CD_TOT',
                      'CN_TOT',
                      'CO_TOT',
                      'COLI_FEC',
                      'COLI_FEC_UFC',
                      'COLI_TOT',
                      'COLI_TOT_UFC',
                      'CONDUC_CAMPO',
                      'CONDUC_CAMPO_FON',
                      'CONDUC_CAMPO_MED',
                      'CONDUC_CAMPO_SUP',
                      'CONDUC_LAB',
                      'DUR_TOT',
                      'E_COLI',
                      'E_COLI_UFC',
                      'FLUORUROS_SOL',
                      'FLUORUROS_TOT',
                      'HAPS',
                      'HAPS_8310',
                      'HG_SOL',
                      'HG_TOT',
                      'HIDROCAR_F_MED_PES',
                      'HIDROCAR_F_PES',
                      'PB_SOL',
                      'PB_TOT',
                      'pH_CAMPO',
                      'pH_CAMPO_FON',
                      'pH_CAMPO_MED',
                      'pH_CAMPO_SUP',
                      'pH_LAB',
                      'PROFUNDIDAD',
                      'SO4_SOL',
                      'SO4_TOT',
                      'TEMP_AGUA',
                      'TEMP_AGUA_0.5',
                      'TEMP_AGUA_1',
                      'TEMP_AGUA_10',
                      'TEMP_AGUA_11',
                      'TEMP_AGUA_12',
                      'TEMP_AGUA_13',
                      'TEMP_AGUA_14',
                      'TEMP_AGUA_15',
                      'TEMP_AGUA_16',
                      'TEMP_AGUA_17',
                      'TEMP_AGUA_18',
                      'TEMP_AGUA_19',
                      'TEMP_AGUA_2',
                      'TEMP_AGUA_20',
                      'TEMP_AGUA_21',
                      'TEMP_AGUA_22',
                      'TEMP_AGUA_23',
                      'TEMP_AGUA_24',
                      'TEMP_AGUA_25',
                      'TEMP_AGUA_26',
                      'TEMP_AGUA_27',
                      'TEMP_AGUA_28',
                      'TEMP_AGUA_29',
                      'TEMP_AGUA_3',
                      'TEMP_AGUA_30',
                      'TEMP_AGUA_31',
                      'TEMP_AGUA_32',
                      'TEMP_AGUA_33',
                      'TEMP_AGUA_34',
                      'TEMP_AGUA_35',
                      'TEMP_AGUA_36',
                      'TEMP_AGUA_37',
                      'TEMP_AGUA_38',
                      'TEMP_AGUA_39',
                      'TEMP_AGUA_39.5',
                      'TEMP_AGUA_4',
                      'TEMP_AGUA_5',
                      'TEMP_AGUA_6',
                      'TEMP_AGUA_7',
                      'TEMP_AGUA_8',
                      'TEMP_AGUA_9',
                      'TEMP_AGUA_FON',
                      'TEMP_AGUA_MED',
                      'TEMP_AGUA_SUP']

# Crear el DataFrame reducido
df_m= df_final[columnas_deseadas_m].copy()
print(df_m.shape)

#(129235, 29) df_s and df_m: (129235, 101)

(129235, 29)

(129235, 101)


In [73]:
# 16 standardize col names in small data set

# remove leading/trailing spaces, convert to lowercase, and replace spaces with underscores, remove accents
#df_s.columns = [c.strip().lower().replace(" ", "_") for c in df_s.columns]
# df_s.columns

def limpiar_columnas(cols):
    nuevas = []
    for c in cols:
        # asegurar string
        c = str(c)
        
        # quitar espacios laterales
        c = c.strip()
        
        # quitar acentos
        c = unicodedata.normalize("NFKD", c)
        c = "".join(ch for ch in c if not unicodedata.combining(ch))
        
        # minusculas
        c = c.lower()
        
        # reemplazar espacios y guiones por _
        c = re.sub(r"[ \-]+", "_", c)
        
        # quitar caracteres raros (opcional pero recomendado)
        c = re.sub(r"[^\w_]", "", c)
        
        nuevas.append(c)
    return nuevas


df_s.columns = limpiar_columnas(df_s.columns)



# verificar que se hayan estandarizado los nombres de las columnas
print(df_s.head())
print(df_s.columns.tolist())
print(df_s.info())

# se observa que la mayoria de las variable cantitativas son ojbect y no float 


       clave_sitio      clave_de_monitoreo     nombre_del_sitio  \
0  BROTE CARMINA 3  BROTE CARMINA 3-131122      CARMINA 3 BROTE   
1        CARMINA 2        CARMINA 2-131122            CARMINA 2   
2         CAZEPA-1         CAZEPA-1-260921  POZO SAN FERNANDO 1   
3         CAZEPA-1         CAZEPA-1-310722  POZO SAN FERNANDO 1   
4         CAZEPA-1         CAZEPA-1-181023  POZO SAN FERNANDO 1   

  tipo_cuerpo_de_agua tipo_cuerpo_simplificado fecha_realizacion   ano  \
0         SUBTERRÁNEO              subterraneo             44873  2022   
1              LÓTICO                   lotico             44873  2022   
2         SUBTERRÁNEO              subterraneo             44462  2021   
3         SUBTERRÁNEO              subterraneo             44769  2022   
4         SUBTERRÁNEO              subterraneo             45217  2023   

        cuenca  clave_acuifero          acuifero  ... caudal fluoruros_sol  \
0          NaN    522.00000000  PRESA LA AMISTAD  ...    NaN           NaN

In [74]:
# 17 corregir la fecha de realizacion esta en un formato object y ademas sus valores no corresponden con las fechas en excel
print(df_s["fecha_realizacion"].head(10))
print(df_s["fecha_realizacion"].dtype) #object columna viene “mixta” (unos números, otros strings) a veces al leer XLSB quedan mezclados: algunos como serial, otros como "11/8/2022" string
df_s["fecha"] = pd.to_datetime(df_s["fecha_realizacion"], errors="coerce")
print()
print(df_s[("fecha")].head())
print

# la fecha esta mal probablemente por los numeros seriales de excel
# fecha original es de forma MM/DD/YYYY

s = df_s["fecha_realizacion"]

# 1) Intento como serial Excel (número de días)
s_num = pd.to_numeric(s, errors="coerce")
fecha_serial = pd.to_datetime(s_num, unit="D", origin="1899-12-30", errors="coerce")

# 2) Intento como texto (en tu ejemplo: m/d/Y)
# (si tu Excel está en formato dd/mm/yyyy cambia el format)
fecha_texto = pd.to_datetime(s.astype(str).str.strip(), format="%m/%d/%Y", errors="coerce")

# 3) Combinar: primero serial, si falta usar texto
df_s["fecha_realizacion_dt"] = fecha_serial.fillna(fecha_texto)
print()
df_s[["fecha_realizacion", "fecha_realizacion_dt"]].head(10)




0    44873
1    44873
2    44462
3    44769
4    45217
5    44463
6    44774
7    45218
8    44463
9    44771
Name: fecha_realizacion, dtype: object
object

0   1970-01-01 00:00:00.000044873
1   1970-01-01 00:00:00.000044873
2   1970-01-01 00:00:00.000044462
3   1970-01-01 00:00:00.000044769
4   1970-01-01 00:00:00.000045217
Name: fecha, dtype: datetime64[ns]



,fecha_realizacion,fecha_realizacion_dt
0,44873,2022-11-08
1,44873,2022-11-08
2,44462,2021-09-23
3,44769,2022-07-27
4,45217,2023-10-18
5,44463,2021-09-24
6,44774,2022-08-01
7,45218,2023-10-19
8,44463,2021-09-24
9,44771,2022-07-29


In [75]:
# 18 fechas de 2012-2024
print('min', df_s["fecha_realizacion_dt"].min())
print()
print('max', df_s["fecha_realizacion_dt"].max())
print()
print(df_s["fecha_realizacion"].dtype)
print(df_s["fecha_realizacion_dt"].dtype) #worked

# Eliminar la variable fecha con la fecha equivocada, axis = 1 para columnas
df_s = df_s.drop('fecha', axis=1)

# checar las variables
df_s.columns.tolist()
# df_s["fecha_realizacion"].head(20).tolist()
#df_s.loc[df_s["fecha_realizacion_dt"].isna(), "fecha_realizacion"].head(30)

min 2012-03-12 00:00:00

max 2024-12-17 00:00:00

object
datetime64[ns]


['clave_sitio',
 'clave_de_monitoreo',
 'nombre_del_sitio',
 'tipo_cuerpo_de_agua',
 'tipo_cuerpo_simplificado',
 'fecha_realizacion',
 'ano',
 'cuenca',
 'clave_acuifero',
 'acuifero',
 'estado',
 'municipio',
 'cuerpo_de_agua',
 'tipo_de_cuerpo_de_agua',
 'tipo_de_cuerpo_simplificado',
 'subtipo_cuerpo_agua',
 'latitud',
 'longitud',
 'profundidad',
 'caudal',
 'fluoruros_sol',
 'fluoruros_tot',
 'as_tot',
 'e_coli_ufc',
 'coli_fec_ufc',
 'coli_tot_ufc',
 'coli_fec',
 'coli_tot',
 'e_coli',
 'fecha_realizacion_dt']

In [76]:
# 19 evalauar como funcionó la conversion. si fallo, tendremos unos NA donde habian fechas en la variable original
# si tenmos NA donde habian 4 fechas en formato DD/MM/YYYY 
df_s.loc[df_s["fecha_realizacion_dt"].isna(), "fecha_realizacion"].head(30)
# corregimos fecha_realizacion y volvemos a converir
# indices problematicos: 16443, 16445, 16451, 16452

# Ver las fechas problemáticas
indices_problematicos = [16443, 16445, 16451, 16452]
print(df_s.loc[indices_problematicos, ['fecha_realizacion', 'fecha_realizacion_dt']])
# los indices tiene formato DD/MM/YYYY no estan en el formato serial que se convierte al formato MM/DD/YYYY
print(df_s["fecha_realizacion"].head()) # otras fechas estan en el formato serial

# Convertir las fechas DD/MM/YYYY a datetime
fechas_corregidas = pd.to_datetime(
    df_s.loc[indices_problematicos, "fecha_realizacion"], 
    format="%d/%m/%Y"  # Formato DD/MM/YYYY
)
print(df_s.loc[indices_problematicos, ['fecha_realizacion', 'fecha_realizacion_dt']])

# Convertir datetime a serial Excel (días desde 1899-12-30)
origen_excel = pd.Timestamp('1899-12-30')
serial_corregido = (fechas_corregidas - origen_excel).dt.days

# Actualizar SOLO esos 4 índices en fecha_realizacion
df_s.loc[indices_problematicos, "fecha_realizacion"] = serial_corregido

print("\nValores después de la corrección formato serial excel:")
print(df_s.loc[indices_problematicos, "fecha_realizacion"])
print(f"\nTipo de datos: {type(df_s.loc[16443, 'fecha_realizacion'])}")
print(df_s.loc[indices_problematicos, ['fecha_realizacion', 'fecha_realizacion_dt']])


      fecha_realizacion fecha_realizacion_dt
16443        18/01/2023                  NaT
16445        18/01/2023                  NaT
16451        18/01/2023                  NaT
16452        18/01/2023                  NaT
0    44873
1    44873
2    44462
3    44769
4    45217
Name: fecha_realizacion, dtype: object
      fecha_realizacion fecha_realizacion_dt
16443        18/01/2023                  NaT
16445        18/01/2023                  NaT
16451        18/01/2023                  NaT
16452        18/01/2023                  NaT

Valores después de la corrección formato serial excel:
16443    44944
16445    44944
16451    44944
16452    44944
Name: fecha_realizacion, dtype: object

Tipo de datos: <class 'int'>
      fecha_realizacion fecha_realizacion_dt
16443             44944                  NaT
16445             44944                  NaT
16451             44944                  NaT
16452             44944                  NaT


In [77]:
# 20 Tu código original funcionará correctamente
s = df_s["fecha_realizacion"]
s_num = pd.to_numeric(s, errors="coerce")
fecha_serial = pd.to_datetime(s_num, unit="D", origin="1899-12-30", errors="coerce")
fecha_texto = pd.to_datetime(s.astype(str).str.strip(), format="%m/%d/%Y", errors="coerce")
df_s["fecha_realizacion_dt"] = fecha_serial.fillna(fecha_texto)
# verificar que ya no hay fechas problemáticas
df_s.loc[df_s["fecha_realizacion_dt"].isna(), "fecha_realizacion"].head(30) # ya no hay fechas problemáticas
# verificar valores de los indices en fecha_realizacion y fecha_realizacion_dt
print(df_s.loc[indices_problematicos, ['fecha_realizacion', 'fecha_realizacion_dt']]) # resuelto


      fecha_realizacion fecha_realizacion_dt
16443             44944           2023-01-18
16445             44944           2023-01-18
16451             44944           2023-01-18
16452             44944           2023-01-18


In [78]:
# 21 ya limpiamos cuerpos de agua y fecha, ahora hay que limpiar contaminantes
print(df_s['as_tot'].dtype)
print(df_s['fluoruros_tot'].dtype)
print(df_s['as_tot'].unique().tolist())
print(df_s['fluoruros_tot'].unique().tolist())
# estas variable son objects y deberian de ser numéricas convertirlas a numéricas si es posible, tienen valors como "<0.01" que no se pueden convertir directamente a números, hay que limpiarlos primero

object
object
['<0.0015', nan, 0.0146, 0.0074, 0.013, 0.0209, 0.01, 0.0096, 0.021, 0.007, 0.0135, 0.0085, 0.0157, 0.0199, 0.0079, 0.0131, 0.0287, 0.016, 0.0243, 0.0205, 0.0239, 0.015, 0.028, 0.011, 0.0211, 0.0308, 0.0227, 0.0322, 0.022, 0.0056, 0.01789, 0.01093, 0.0107, 0.0273, 0.0072, 0.0127, 0.0122, 0.0245, 0.017, 0.0112, 0.0505, 0.0119, 0.0097, 0.0246, 0.02088, 0.0216, 0.0183, 0.0139, 0.01554, 0.00935, 0.0129, 0.0123, 0.0269, 0.0148, 0.019, 0.006, 0.0092, 0.0132, 0.0116, 0.2803, 0.0203, 0.0141, 0.0145, 0.01996, 0.01721, 0.0136, 0.0111, 0.0185, 0.006557, 0.00605, 0.0121, 0.0099, 0.0086, 0.0128, 0.0115, 0.00915, 0.07, 0.096, 0.0105, 0.0109, 0.0082, 0.0114, 0.0087, 0.0075, 0.0101, 0.0095, 0.0094, 0.0066, 0.0117, 0.0104, 0.009, 0.00858, 0.00862, 0.0083, 0.004, 0.0177, 0.0175, 0.0143, 0.005, 0.0089, 0.0158, 0.0186, 0.0316, 0.01262, 0.0068, 0.00785, 0.0155, 0.01108, 0.0137, 0.0152, 0.0208, 0.0045, 0.008, 0.0576, 0.0108, 0.0124, 0.0176, 0.0168, 0.01592, 0.01035, 0.0144, 0.0182, 0.0163, 0.0

In [79]:
# 22 crear funcion de limpieza para convertir valores censurados en texto a numéricos
def limpiar_variable_numerica(valor):
    """
    Convierte valores censurados a numéricos
    Para AS_TOT
    < 0.010	=	0.00999
    <0.00017	=	0.00016999
    <0.0007	=	0.0006999
    <0.0015	=	0.0014999
    <0.002	=	0.001999
    <0.0025	=	0.0024999
    <0.005	=	0.004999
    <0.01	=	0.00999
    <0.02	=	0.001999
    <5	=	4.999 este caso parece un error de tipeo quizas >5 tenia mas sentido dado que el limite es 0.01mg/L en grandes ciudade y 0.025mg/L en pueblitos de menos de 50k

    Para FLUORUROS_TOT
    < 0.2
    <0.01
    <0.02
    <0.03
    <0.0364
    <0.2
    <2  este case parece ser un error de tipeo quizas >2 tendria mas sentido dado que el limite es 1mg/L en grandes ciudade y 1.5mg/L en pueblitos de menos de 50k
    >2

    """
    if pd.isna(valor):  # Manejar valores nulos
        return valor
    
    # Convertir el valor a string y elimina sus espacio 
    valor = str(valor).strip()
    
    # Manejar casos especiales con <, quitar <, convertir  numerico, y cambiar el valor a un número ligeramente menor
    if valor.startswith('<'):
        # Para <0.010, tomar 0.00999 (valor ligeramente menor)
        numero = valor.replace('<', '')
        return float(numero) * 0.99999
    
    elif valor.startswith('>'):
        # Para >5, tomar 5.001 (valor ligeramente mayor)
        numero = valor.replace('>', '')
        return float(numero) * 1.001
    
    else:
        # Intentar convertir directamente a float
        try:
            return float(valor)
        except:
            return np.nan  # Si no se puede convertir, devolver NaN

# Aplicar la limpieza a as_tot y fluoruros_tot
df_s['as_tot_float'] = df_s['as_tot'].apply(limpiar_variable_numerica)
df_s['fluoruros_tot_float'] = df_s['fluoruros_tot'].apply(limpiar_variable_numerica)


# Verificar los resultados
print("=== ANTES Y DESPUÉS DE LA LIMPIEZA ===")
print("\nPrimeras 10 filas de as_tot:")
print(df_s[['as_tot', 'as_tot_float']].head(10))
print('formato as_tot_float:', df_s['as_tot_float'].dtype) #float64 now


print("\nPrimeras 10 filas de fluoruros_tot:")
print(df_s[['fluoruros_tot', 'fluoruros_tot_float']].head(10))
print('formato fluoruros_tot_float:', df_s['fluoruros_tot_float'].dtype)



=== ANTES Y DESPUÉS DE LA LIMPIEZA ===

Primeras 10 filas de as_tot:
      as_tot  as_tot_float
0    <0.0015    0.00149999
1        NaN           NaN
2 0.01460000    0.01460000
3 0.00740000    0.00740000
4 0.01300000    0.01300000
5 0.02090000    0.02090000
6 0.01000000    0.01000000
7 0.00740000    0.00740000
8 0.00960000    0.00960000
9 0.02100000    0.02100000
formato as_tot_float: float64

Primeras 10 filas de fluoruros_tot:
  fluoruros_tot  fluoruros_tot_float
0    0.82000000           0.82000000
1           NaN                  NaN
2    0.46920000           0.46920000
3    0.52000000           0.52000000
4    0.77000000           0.77000000
5    0.35760000           0.35760000
6    0.43000000           0.43000000
7    0.50000000           0.50000000
8    0.31690000           0.31690000
9    0.50000000           0.50000000
formato fluoruros_tot_float: float64


In [80]:
# 23 Configurar pandas para mostrar más decimales
pd.set_option('display.precision', 8)  # Mostrar 8 decimales
pd.set_option('display.float_format', '{:.8f}'.format)  # Formato específico

# Ahora aplica tu función
df_s['as_tot_float'] = df_s['as_tot'].apply(limpiar_variable_numerica)
df_s['fluoruros_tot_float'] = df_s['fluoruros_tot'].apply(limpiar_variable_numerica)

# Verificar los resultados con más precisión
print("=== RESULTADOS CON MÁS DECIMALES ===")
print(df_s[['as_tot', 'as_tot_float']].sample(20))
print()
print(df_s[['fluoruros_tot', 'fluoruros_tot_float']].head(40))

=== RESULTADOS CON MÁS DECIMALES ===
           as_tot  as_tot_float
71016     <0.0015    0.00149999
58898         NaN           NaN
123962    <0.0015    0.00149999
50894     <0.0015    0.00149999
5467      <0.0015    0.00149999
67812     <0.0015    0.00149999
99409  0.01300000    0.01300000
125012    <0.0015    0.00149999
85991     <0.0015    0.00149999
1985          NaN           NaN
49306     <0.0015    0.00149999
51658     <0.0015    0.00149999
33151     <0.0015    0.00149999
70817     <0.0015    0.00149999
60531     <0.0015    0.00149999
124306    <0.0015    0.00149999
53582         NaN           NaN
35191     <0.0015    0.00149999
109923 0.00950000    0.00950000
57013     <0.0015    0.00149999

   fluoruros_tot  fluoruros_tot_float
0     0.82000000           0.82000000
1            NaN                  NaN
2     0.46920000           0.46920000
3     0.52000000           0.52000000
4     0.77000000           0.77000000
5     0.35760000           0.35760000
6     0.43000000        

In [81]:
# 24 Verificar AS y F valores que empiezan con '<'
print("\n=== VALORES CENSURADOS CON '<' ===")
mask_menor = df_s['as_tot'].astype(str).str.startswith('<')
if mask_menor.any():
    print("as_tot con '<':")
    print(df_s.loc[mask_menor, ['as_tot', 'as_tot_float']].head())

mask_menor_f = df_s['fluoruros_tot'].astype(str).str.startswith('<')
if mask_menor_f.any():
    print("fluoruros_tot con '<':")
    print(df_s.loc[mask_menor_f, ['fluoruros_tot', 'fluoruros_tot_float']].head())

# Verificar AS y F valores que empiezan con '>'
print("\n=== VALORES CENSURADOS CON '>' ===")
mask_mayor = df_s['fluoruros_tot'].astype(str).str.startswith('>')
if mask_mayor.any():
    print("fluoruros_tot con '>':")
    print(df_s.loc[mask_mayor, ['fluoruros_tot', 'fluoruros_tot_float']].head())


=== VALORES CENSURADOS CON '<' ===
as_tot con '<':
     as_tot  as_tot_float
0   <0.0015    0.00149999
28  <0.0015    0.00149999
42  <0.0015    0.00149999
43  <0.0015    0.00149999
44  <0.0015    0.00149999
fluoruros_tot con '<':
     fluoruros_tot  fluoruros_tot_float
811        <0.0364           0.03639964
931        <0.0364           0.03639964
1020       <0.0364           0.03639964
1063       <0.0364           0.03639964
1267       <0.0364           0.03639964

=== VALORES CENSURADOS CON '>' ===
fluoruros_tot con '>':
     fluoruros_tot  fluoruros_tot_float
32              >2           2.00200000
479             >2           2.00200000
1019            >2           2.00200000


In [82]:
# 25 Estadísticas descriptivas de ambas variables limpias
estadisticas = df_s[['as_tot_float', 'fluoruros_tot_float']].describe()
print("=== ESTADÍSTICAS DESCRIPTIVAS ===")
print(estadisticas)


=== ESTADÍSTICAS DESCRIPTIVAS ===
         as_tot_float  fluoruros_tot_float
count 117742.00000000       13693.00000000
mean       0.00688241           1.05505524
std        0.04293046           1.83736109
min        0.00010000           0.00999990
25%        0.00149999           0.28380000
50%        0.00149999           0.55000000
75%        0.00149999           1.14000000
max        6.83300000          45.21320000


In [83]:
# 26 Ahora vamos a crear variables indicadores para indicar que muestras o fuentes de agua tienen contaminantes por arribas de los niveles permisible en el agua para consumo humano
# segun la NOM-127-SSA1-2021, el límite permisible para arsénico es de 0.01 mg/L en grandes ciudades y 0.025mg en pueblos de menos de 50k person hasta 2027 y 
# para fluoruros es de 1 mg/L en grandes ciudades y 1.5 mg/L en pueblitos de menos de 50k habitantes, pero 
# como no tenemos información sobre el tamaño del municipio, 
# vamos a usar el límite más estricto para ambos contaminantes (0.01 mg/L para arsénico y 1 mg/L para fluoruros) para crear las variables indicadoras
# 
# # Crear variable indicadora para Arsénico (límite > 0.01)
df_s['supNOM127SSA12021_As'] = df_s['as_tot_float'].apply(
    lambda x: 1 if pd.notna(x) and x > 0.01 else (0 if pd.notna(x) else np.nan)
)

# Crear variable indicadora para Fluoruros (límite > 1)
df_s['supNOM127SSA12021_F'] = df_s['fluoruros_tot_float'].apply(
    lambda x: 1 if pd.notna(x) and x > 1 else (0 if pd.notna(x) else np.nan)
)

In [84]:
# 27 Verificar las nuevas variables
print("=== VERIFICACIÓN DE VARIABLES INDICADORAS ===")
print("\nPrimeras 10 filas:")
print(df_s[['as_tot_float', 'supNOM127SSA12021_As', 'fluoruros_tot_float', 'supNOM127SSA12021_F']].head(10))

# Contar valores por categoría
print(f"\nArsénico - Valores que superan límite: {df_s['supNOM127SSA12021_As'].sum()}")
print(f"\nArsénico - Valores no ausentes: {df_s['as_tot_float'].notna().sum()}")

print()
print(f"Fluoruros - Valores que superan límite: {df_s['supNOM127SSA12021_F'].sum()}")
print(f"\nFluoruros - Valores no ausentes: {df_s['fluoruros_tot_float'].notna().sum()}")


=== VERIFICACIÓN DE VARIABLES INDICADORAS ===

Primeras 10 filas:
   as_tot_float  supNOM127SSA12021_As  fluoruros_tot_float  \
0    0.00149999            0.00000000           0.82000000   
1           NaN                   NaN                  NaN   
2    0.01460000            1.00000000           0.46920000   
3    0.00740000            0.00000000           0.52000000   
4    0.01300000            1.00000000           0.77000000   
5    0.02090000            1.00000000           0.35760000   
6    0.01000000            0.00000000           0.43000000   
7    0.00740000            0.00000000           0.50000000   
8    0.00960000            0.00000000           0.31690000   
9    0.02100000            1.00000000           0.50000000   

   supNOM127SSA12021_F  
0           0.00000000  
1                  NaN  
2           0.00000000  
3           0.00000000  
4           0.00000000  
5           0.00000000  
6           0.00000000  
7           0.00000000  
8           0.00000000  
9

In [85]:
# 28 Calcular proporciones para cada contaminante
print("=== PROPORCIONES DE MUESTRAS QUE SUPERAN LÍMITES ===")

# Arsénico (límite > 0.01 mg/L)
total_as_validas = df_s['supNOM127SSA12021_As'].notna().sum()
superan_as = df_s['supNOM127SSA12021_As'].sum()
proporcion_as = superan_as / total_as_validas

print(f"Arsénico:")
print(f"  - Muestras válidas: {total_as_validas}")
print(f"  - Muestras que superan límite: {superan_as}")
print(f"  - Proporción: {proporcion_as:.4f} ({proporcion_as*100:.2f}%)")

# Fluoruros (límite > 1 mg/L)
total_f_validas = df_s['supNOM127SSA12021_F'].notna().sum()
superan_f = df_s['supNOM127SSA12021_F'].sum()
proporcion_f = superan_f / total_f_validas

print(f"\nFluoruros:")
print(f"  - Muestras válidas: {total_f_validas}")
print(f"  - Muestras que superan límite: {superan_f}")
print(f"  - Proporción: {proporcion_f:.4f} ({proporcion_f*100:.2f}%)")

=== PROPORCIONES DE MUESTRAS QUE SUPERAN LÍMITES ===
Arsénico:
  - Muestras válidas: 117742
  - Muestras que superan límite: 14056.0
  - Proporción: 0.1194 (11.94%)

Fluoruros:
  - Muestras válidas: 13693
  - Muestras que superan límite: 3933.0
  - Proporción: 0.2872 (28.72%)


In [86]:
# 29 Muestras que superan CUALQUIERA de los dos límites
df_s['supera_cualquier_limite'] = (
    (df_s['supNOM127SSA12021_As'] == 1) | 
    (df_s['supNOM127SSA12021_F'] == 1)
).astype('Int64')  # Usar Int64 para permitir NA")

# Calcular proporción combinada
total_combinadas = ((df_s['supNOM127SSA12021_As'].notna()) | 
                   (df_s['supNOM127SSA12021_F'].notna())).sum()
superan_cualquiera = df_s['supera_cualquier_limite'].sum()
proporcion_combinada = superan_cualquiera / total_combinadas

print(f"\nCombinado (As O F):")
print(f"  - Muestras con al menos un valor válido: {total_combinadas}")
print(f"  - Muestras que superan cualquier límite: {superan_cualquiera}")
print(f"  - Proporción de casos que supera cualquier límite: {proporcion_combinada:.4f} ({proporcion_combinada*100:.2f}%)")



Combinado (As O F):
  - Muestras con al menos un valor válido: 118358
  - Muestras que superan cualquier límite: 15926
  - Proporción de casos que supera cualquier límite: 0.1346 (13.46%)


In [87]:
# 30 Convertir indicadores a Int64 (con mayúscula) que permite valores nulos
print("Tipo de dato antes de la conversión F:", df_s['supNOM127SSA12021_F'].dtype)
print("Tipo de dato antes de la conversión As:", df_s['supNOM127SSA12021_As'].dtype)
df_s['supNOM127SSA12021_F'] = df_s['supNOM127SSA12021_F'].astype('Int64')
df_s['supNOM127SSA12021_As'] = df_s['supNOM127SSA12021_As'].astype('Int64')

# Verificar el cambio
print("Tipo de dato después de la conversión F :")
print(df_s['supNOM127SSA12021_F'].dtype)
print("\nPrimeros valores:")
print(df_s['supNOM127SSA12021_F'].head(10))

# Verificar el cambio
print("Tipo de dato después de la conversión As:")
print(df_s['supNOM127SSA12021_As'].dtype)
print("\nPrimeros valores:")
print(df_s['supNOM127SSA12021_As'].head(10))

Tipo de dato antes de la conversión F: float64
Tipo de dato antes de la conversión As: float64
Tipo de dato después de la conversión F :
Int64

Primeros valores:
0       0
1    <NA>
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
Name: supNOM127SSA12021_F, dtype: Int64
Tipo de dato después de la conversión As:
Int64

Primeros valores:
0       0
1    <NA>
2       1
3       0
4       1
5       1
6       0
7       0
8       0
9       1
Name: supNOM127SSA12021_As, dtype: Int64


In [88]:
# 31 checar var en la base y quitar variable no necesarias
df_s.info() # todo las variables transformadas estan en fomato esperado

# encontre 2 variables que ya no necesito porque ya tengo las versiones limpias de as_tot y fluoruros_tot
#df_s = df_s.drop('as_tot_limpio', axis=1)
#df_s = df_s.drop('fluoruros_tot_limpio', axis=1)
#print(df_s.columns)

# Guardar el DataFrame limpio en un nuevo archivo CSV   - NO MEJOR GUARDARLO EN OTRO FORMATO PARA PRESERVAR LOS TIPOS DE DATOS Y EVITAR PROBLEMAS DE CODIFICACIÓN
# df_s.to_csv('../tlm_conagua_limpio.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129235 entries, 0 to 129234
Data columns (total 35 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   clave_sitio                  129235 non-null  object        
 1   clave_de_monitoreo           129235 non-null  object        
 2   nombre_del_sitio             129235 non-null  object        
 3   tipo_cuerpo_de_agua          129235 non-null  object        
 4   tipo_cuerpo_simplificado     129235 non-null  object        
 5   fecha_realizacion            129235 non-null  object        
 6   ano                          129235 non-null  int64         
 7   cuenca                       115256 non-null  object        
 8   clave_acuifero               14164 non-null   float64       
 9   acuifero                     14164 non-null   object        
 10  estado                       129235 non-null  object        
 11  municipio                 

In [89]:
# 32 parquet requires single consistent dtype per col

# id problematic columns
bad_cols = []
for c in df_s.columns:
    if df_s[c].dtype == 'object':
        # look for inequality-coded strings
        if df_s[c].astype(str).str.contains(r'^\s*[<>]', regex=True, na=False).any():
            bad_cols.append(c)

bad_cols

['profundidad',
 'caudal',
 'fluoruros_sol',
 'fluoruros_tot',
 'as_tot',
 'coli_fec_ufc',
 'coli_tot_ufc',
 'coli_fec',
 'coli_tot',
 'e_coli']

In [90]:
# 33 look up string value location in quant vars bad_cols

for col in bad_cols:
    mask = df_s[col].astype(str).str.contains("3X101", na=False)
    if mask.any():
        print(f"\nEncontrado en columna: {col}")
        print(df_s.loc[mask, col].head())

# 3X101 found in coli_fec and coli_tot
# 👉 Coliformes fecales < 3 × 10¹ NMP/100 mL
# 👉 i.e. < 30 NMP/100 mL


Encontrado en columna: coli_fec
128065    <3X101
128068    <3X101
128070    <3X101
128071    <3X101
128099    <3X101
Name: coli_fec, dtype: object

Encontrado en columna: coli_tot
128065    <3X101
128068    <3X101
128070    <3X101
128071    <3X101
128099    <3X101
Name: coli_tot, dtype: object


NOTES 

1️⃣ DL − ε (e.g., 29.999 when DL=30)
Conservative upper-bound substitution
Key references
EPA / risk-assessment guidance
The US EPA describes assigning detection-limit values as a conservative approach that “produces a mean concentration biased high.”
This corresponds to your DL − ε or DL substitution strategy:
👉 Used when you want a conservative public-health estimate
👉 Common in regulatory exposure assessment.

Helsel (environmental statistics textbook)
Notes that substitution approaches like DL/2 or related constants have historically been used in environmental chemistry datasets.

Use DL − ε when: 
doing compliance screening
avoiding underestimation of contamination
reporting upper-bound exposure.

Typical in: groundwater risk assessment, microbiological monitoring, 
arsenic/fluoride compliance analyses.

2️⃣ DL/2 substitution (e.g., 15 when DL=30)
Traditional environmental statistics approach
Key references
Classic environmental statistics practice
Substitution of half the detection limit is cited as a “most common approach” for nondetect environmental data.
Simulation study evidence
Threshold/2 substitution often produced less biased estimates under certain censoring scenarios.

This is why DL/2 is standard in: descriptive environmental monitoring, water-quality trend analyses, preliminary epidemiologic studies.

DL/2 is appropriate when:
censoring proportion is moderate (<40–50%)
distribution roughly lognormal
you need quick summary statistics.

It’s not ideal for inference but acceptable for EDA.

3️⃣ Censored statistical models (recommended for inference) This is the recommended method academically.
Best scientific practice
Maximum likelihood estimation (MLE)
Kaplan–Meier methods
Regression on order statistics (ROS)
Tobit/interval regression.

Key references
Definitive textbook (gold standard)
Helsel, D.R. (2012)
Statistics for Censored Environmental Data Using Minitab and R.
Introduces survival-analysis methods for environmental censored data.
Widely cited in water-quality, hydrochemistry, and exposure science.

Use censored models when:
you want unbiased parameter estimates
performing regression/trend analysis
detection limits vary across samples
publication-quality inference is needed.

⭐ Additional foundational references (important)
Detection limits and environmental monitoring context
Environmental data frequently contain values below detection limits; such values represent censoring rather than true zero concentration.

⭐ Recommended citation set for your research
Core textbook
Helsel, D.R. (2012).
Statistics for Censored Environmental Data Using Minitab and R.
Substitution methods
EPA Regional Guidance on Detection Limits.
Clarke (1998) environmental censored data simulation study.
Modern statistical approaches
Canales et al. (2018) environmental microbiology censored data analysis.
Food/environment nondetect statistical methods review.
These cover regulatory, statistical, and applied perspectives.
⭐ My recommendation for your specific water-quality project
Given:
arsenic/fluoride hydrochemistry
microbiological NMP values
environmental policy angle
Best practice:
EDA phase → DL/2 substitution acceptable.
Compliance / risk reporting → DL − ε conservative.
Research paper / causal analysis → censored models (ROS, MLE, survival analysis).

That aligns with environmental statistics norms.

In [91]:
# 34 clean up coli_fec and coli_tot using a custom parser for MPN detection limits
def parse_mpn_detection_limit(s: str):
    """
    Parse detection limit expressions used in MPN/NMP microbiology data.

    Returns a float DL if the string encodes a detection limit, otherwise None.

    Handles:
      - "3X101" meaning 3 * 10^1 = 30
      - "3x10^1" meaning 3 * 10^1 = 30
      - "3 x 10 1" meaning 3 * 10^1 = 30
      - "30" meaning 30
    """
    if s is None:
        return None

    s = str(s).strip()
    if s == "" or s.lower() in {"nan", "na", "nd", "n/d"}:
        return None

    # normalize variants: ×, commas, spaces
    s_norm = s.replace("×", "x").replace(",", ".")
    s_norm = re.sub(r"\s+", "", s_norm)  # remove all whitespace

    # Case 1: pure numeric like "30" or "30.0"
    if re.fullmatch(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", s_norm):
        try:
            return float(s_norm)
        except:
            return None

    # Case 2: "3x101" or "3X101" meaning 3 * 10^1
    m = re.fullmatch(r"(?P<a>\d*\.?\d+)x10(?P<b>-?\d+)", s_norm, flags=re.IGNORECASE)
    if m:
        a = float(m.group("a"))
        b = int(m.group("b"))
        return a * (10 ** b)

    # Case 3: "3x10^1"
    m = re.fullmatch(r"(?P<a>\d*\.?\d+)x10\^(?P<b>-?\d+)", s_norm, flags=re.IGNORECASE)
    if m:
        a = float(m.group("a"))
        b = int(m.group("b"))
        return a * (10 ** b)

    return None


def conservative_upper_bound(valor, eps_rel=1e-6, eps_abs=1e-12):
    """
    Conservative upper bound method (DL - ε):
      - If value begins with '<', interpret the DL and return DL - ε
      - If value is numeric, return numeric
      - Otherwise return NaN

    ε chosen as max(eps_abs, eps_rel * DL) to ensure strictly below DL.
    """
    if pd.isna(valor):
        return np.nan

    s = str(valor).strip()
    if s == "":
        return np.nan

    # detect censoring
    if s.startswith("<"):
        inner = s[1:].strip()
        dl = parse_mpn_detection_limit(inner)
        if dl is None:
            return np.nan
        eps = max(eps_abs, eps_rel * dl)
        return dl - eps

    # (optional) handle > as just numeric lower bound; here we parse but do not inflate
    if s.startswith(">"):
        inner = s[1:].strip()
        dl = parse_mpn_detection_limit(inner)
        return dl if dl is not None else np.nan

    # otherwise: try numeric parse
    dl = parse_mpn_detection_limit(s)
    return dl if dl is not None else np.nan


# Apply to your two columns
for col in ["coli_fec", "coli_tot"]:
    # keep raw as string (optional but recommended)
    df_s[col] = df_s[col].astype("string")

    # conservative upper bound numeric column
    df_s[f"{col}_ub"] = df_s[col].apply(conservative_upper_bound).astype("float64")

# Quick sanity check: show the censored cases and what they became
mask = (
    df_s["coli_fec"].astype(str).str.strip().str.startswith("<", na=False)
    | df_s["coli_tot"].astype(str).str.strip().str.startswith("<", na=False)
)
df_s.loc[mask, ["coli_fec", "coli_fec_ub", "coli_tot", "coli_tot_ub"]].head(20)


,coli_fec,coli_fec_ub,coli_tot,coli_tot_ub
0,97,97.00000000,<NA>,NaN
1,1076,1076.00000000,<NA>,NaN
2,<10,9.99999000,<NA>,NaN
3,<10,9.99999000,<NA>,NaN
4,<NA>,NaN,<NA>,NaN
5,10,10.00000000,<NA>,NaN
6,<10,9.99999000,<NA>,NaN
7,<NA>,NaN,<NA>,NaN
8,<10,9.99999000,<NA>,NaN
9,<10,9.99999000,<NA>,NaN


In [92]:
# 35 try new cleaning function

def limpiar_variable_numerica(valor):
    """
    Limpia variables numéricas ambientales con posibles censuras (<, >)
    y comas como separador decimal o de miles.

    Devuelve:
        float o np.nan
    """

    if pd.isna(valor):
        return np.nan

    valor = str(valor).strip()

    # --- Handle commas ---

    # Thousands separator: 1,200 → 1200
    if re.match(r'^\d{1,3}(,\d{3})+$', valor):
        valor = valor.replace(',', '')

    # Decimal comma: 0,25 → 0.25
    elif ',' in valor and '.' not in valor:
        valor = valor.replace(',', '.')

    # Normalizar separadores decimales:
    # Caso europeo/latino: 0,25 -> 0.25 ...not sure this will work as supposed to
    #if ',' in valor and '.' not in valor:
    #    valor = valor.replace(',', '.')

    # Caso miles: 1,200 -> 1200 but 1,200 does not have a decimal point...not sure this will work as supposed to
    #elif ',' in valor and '.' in valor:
    #    valor = valor.replace(',', '')

    # --- Detectar censura --- 
    flag = None
    if valor.startswith('<'):
        flag = '<'
        valor = valor[1:].strip()
    elif valor.startswith('>'):
        flag = '>'
        valor = valor[1:].strip()

    # Extraer número válido (evita errores como "3X101")
    # match = re.search(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?', valor)
    # if not match:
    #    return np.nan

    # --- Ignore scientific-style microbiological codes like 3X101 ---
    # (you said these are already handled separately)
    #if re.search(r'\d+[xX]\d+', valor):
    #    return np.nan  # or simply return valor if you prefer untouched

    #numero = float(match.group())

 # Convert numeric safely
    try:
        numero = float(valor)
    except:
        return np.nan
    
    # Aplicar ajuste conservador
    if flag == '<':
        return numero * 0.99999
    elif flag == '>':
        return numero * 1.001
    else:
        return numero

# try it out on profundidad


s = df_s['profundidad'].astype(str).str.strip()

df_s['profundidad_float'] = df_s['profundidad'].apply(limpiar_variable_numerica) #.astype('float64') not required function converts to float already

# Optional: keep a flag for inequality values
df_s['profundidad_flag'] = np.where(s.str.contains(r'[<>]'), s.str.extract(r'([<>])')[0], pd.NA)

# If you want to overwrite the original profundidad with numeric or save original as string
#df_s['profundidad'] = df_s['profundidad_num']
#df_s = df_s.drop(columns=['profundidad_num'])

print(df_s[['profundidad', 'profundidad_float', 'profundidad_flag']].sample(40))


       profundidad  profundidad_float profundidad_flag
119042           3         3.00000000             <NA>
102311         NaN                NaN             <NA>
5337           NaN                NaN             <NA>
26427          NaN                NaN             <NA>
48323            1         1.00000000             <NA>
42071   0.80000000         0.80000000             <NA>
59811          NaN                NaN             <NA>
4437           NaN                NaN             <NA>
34261          NaN                NaN             <NA>
73350   0.80000000         0.80000000             <NA>
37555          NaN                NaN             <NA>
18883   0.70000000         0.70000000             <NA>
36046          NaN                NaN             <NA>
866            NaN                NaN             <NA>
66194          NaN                NaN             <NA>
61422          NaN                NaN             <NA>
111808         NaN                NaN             <NA>
93047     

In [98]:
# 36 limpiar las columnas problemáticas antes de guardar + convertir las columnas problemáticas a string para evitar problemas al guardar en parquet, there is a prob with a 3X101
bad_cols = ['profundidad',
 'caudal',
 'fluoruros_sol',
 #'fluoruros_tot',
 #'as_tot',
 'coli_fec_ufc',
 'coli_tot_ufc',
 #'coli_fec',
 #'coli_tot',
 'e_coli']

for col in bad_cols:
    df_s[col + '_float'] = df_s[col].apply(limpiar_variable_numerica) # astype('float64') # still getting errors cuz of unexpected values like 3X101 NAType
    df_s[col] = df_s[col].astype('string')
# Verificar que las columnas problemáticas se hayan limpiado y convertido correctamente
#    print(f"Columna {col} procesada correctamente.")
#    print(df_s[col + '_float'].describe())

df_s["fluoruros_tot"] = df_s["fluoruros_tot"].astype("string")
df_s["as_tot"] = df_s["as_tot"].astype("string")
df_s["e_coli_ufc"] = df_s["e_coli_ufc"].astype("string")

print(df_s.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129235 entries, 0 to 129234
Data columns (total 44 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   clave_sitio                  129235 non-null  object        
 1   clave_de_monitoreo           129235 non-null  object        
 2   nombre_del_sitio             129235 non-null  object        
 3   tipo_cuerpo_de_agua          129235 non-null  object        
 4   tipo_cuerpo_simplificado     129235 non-null  object        
 5   fecha_realizacion            129235 non-null  object        
 6   ano                          129235 non-null  int64         
 7   cuenca                       115256 non-null  object        
 8   clave_acuifero               14164 non-null   float64       
 9   acuifero                     14164 non-null   object        
 10  estado                       129235 non-null  object        
 11  municipio                 

In [ ]:
# save cleaned data set where data types matter
df_s.to_parquet('../tlm_conagua_limpio.parquet', index=False)
# no funciono varias veces por variable con tipos de valores distintos  
